In [ ]:
# 1) Random sampling (torch.randn, torch.rand, torch.distributions module)
# 2) Demonstration of central limit theorem through simulation
# 3) Autodiff and computational graph construction (grad fields for tensors, backward method (and how it discards the implicitly created computational graph), optimizers, loss functions)
# 4) Application of autodiff to a very simple problem (like fitting a normal and/or a beta distribution through maximum likelihood)

In [1]:
import torch
import numpy as np

# Bernoulli MLE example

In [59]:
trueProb = 0.23
bern = torch.distributions.bernoulli.Bernoulli(torch.tensor([trueProb]))
X_bin = torch.autograd.Variable(bern.sample(sample_shape = [100,]))
X_bin = bern.sample(sample_shape = [100,])
print(X_bin.mean())

tensor(0.2100)


In [49]:
learnedProb = torch.autograd.Variable(torch.rand(1), requires_grad = True)
print(learnedProb)

tensor([0.1430], requires_grad=True)


In [51]:
learningRate = 0.00001
# learnedProb.zero_()
for myIter in range(1000):
    lossFunc = -torch.sum(torch.log(X_bin*learnedProb + (1-X_bin)*(1-learnedProb)))
    lossFunc.backward()
    if myIter % 100 == 0:
        print("log likelihood: {}, learned prob = {}, update = {}".format(lossFunc.data.numpy(),
                                                                          learnedProb.data.numpy(),
                                                                          learnedProb.grad.data.numpy))
    learnedProb.data = learnedProb.data - learnedProb.grad.data*learningRate
    learnedProb.grad.data.zero_()

log likelihood: 53.28410339355469, learned prob = [0.26722452], update = <built-in method numpy of Tensor object at 0x7fb155589d38>
log likelihood: 52.96193313598633, learned prob = [0.25148225], update = <built-in method numpy of Tensor object at 0x7fb1554ee798>
log likelihood: 52.784645080566406, learned prob = [0.23828968], update = <built-in method numpy of Tensor object at 0x7fb1554ee678>
log likelihood: 52.72193908691406, learned prob = [0.23045349], update = <built-in method numpy of Tensor object at 0x7fb1554ee678>
log likelihood: 52.70085144042969, learned prob = [0.22591306], update = <built-in method numpy of Tensor object at 0x7fb1554ee678>
log likelihood: 52.69398880004883, learned prob = [0.22332391], update = <built-in method numpy of Tensor object at 0x7fb1554ee678>
log likelihood: 52.69180679321289, learned prob = [0.22186166], update = <built-in method numpy of Tensor object at 0x7fb1554ee678>
log likelihood: 52.69111633300781, learned prob = [0.22104053], update = <b

In [45]:
print(lossFunc)

tensor(52.6908, grad_fn=<NegBackward>)


# Normal distribution MLE example

In [122]:
trueMu = 1.491
trueSig = 0.1876
normalDist = torch.distributions.normal.Normal(loc = trueMu, scale = trueSig)
# X_bin = torch.autograd.Variable(bern.sample(sample_shape = [100,]))
X_norm = torch.autograd.Variable(normalDist.sample(sample_shape = [100,]))
print(X_norm.mean())
print(np.std(torch.Tensor.numpy(X_norm)))

tensor(1.5275)
0.20134597


In [123]:
learnedMu = torch.autograd.Variable(torch.rand(1), requires_grad = True)
learnedSigma = torch.autograd.Variable(torch.rand(1), requires_grad = True)
print(learnedMu)
print(leanedSigma)

tensor([0.3247], requires_grad=True)
tensor([0.4949], requires_grad=True)


In [124]:
learningRate = 0.000009
# learnedProb.zero_()
n = X_norm.shape[0]
for myIter in range(1000):
    lossFunc = ((n/2)*torch.log(2*np.pi*(learnedSigma**2)) + (1/(2*(learnedSigma**2)))*torch.sum((X_norm - learnedMu)**2))
    lossFunc.backward()
    if myIter % 100 == 0:
        print("log likelihood: {}, learned mu = {}, learned sigma {}".format(lossFunc.data.numpy(),
                                                                          learnedMu.data.numpy(),
                                                                          learnedSigma.data.numpy()))
    learnedMu.data = learnedMu.data - learnedMu.grad.data*learningRate
    learnedSigma.data = learnedSigma.data - learnedSigma.grad.data*learningRate
    learnedMu.grad.data.zero_()
    learnedSigma.grad.data.zero_()

log likelihood: [528.74347], learned mu = [0.32467383], learned sigma [0.37262374]
log likelihood: [147.85068], learned mu = [0.5823783], learned sigma [0.7317697]
log likelihood: [124.17253], learned mu = [0.7185364], learned sigma [0.7791844]
log likelihood: [110.40022], learned mu = [0.8294394], learned sigma [0.77866924]
log likelihood: [98.75214], learned mu = [0.92838806], learned sigma [0.7531023]
log likelihood: [87.07585], learned mu = [1.0209675], learned sigma [0.70938164]
log likelihood: [74.14524], learned mu = [1.1105324], learned sigma [0.64946795]
log likelihood: [58.698563], learned mu = [1.1997578], learned sigma [0.57261634]
log likelihood: [38.86735], learned mu = [1.2913963], learned sigma [0.47569034]
log likelihood: [12.012688], learned mu = [1.3882916], learned sigma [0.35441273]


In [125]:
n = X_norm.shape[0]

In [69]:
n

100

In [72]:
np.pi

3.141592653589793